In [1]:
import pandas as pd

# Анализ данных

In [2]:
fields = ['Product', 'Summary']
data = pd.read_csv("bugs-2018-04-05.csv", usecols=fields)
data.head()

,Product,Summary
0,Thunderbird,e-mail tag is not visible when thread is colla...
1,Calendar,Make use of Assert.jsm in xpcshell tests
2,Core,Add touch-action regions to the layer EventReg...
3,Core,Transition more fields of FrameMetrics to use ...
4,Calendar,Getter fails in calender-migration-dialog on f...


In [3]:
data.describe()

,Product,Summary
count,10000,10000
unique,78,9952
top,Core,reboots-scl1
freq,2122,8


In [4]:
import re
def clean_text(text):
    text = text.replace(r"[^A-Za-z0-9]", " ")
    text = re.sub("[\]\[_'\\\/.,;:\-()<>\"]", " ", text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if len(word)>2)
    return text

data['CleanSummary'] = data.apply(lambda x: clean_text(x['Summary']), axis=1)

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

data['Tokens'] = data['CleanSummary'].apply(tokenizer.tokenize)
data.to_pickle('dataframe.pkl')

data.tail()


,Product,Summary,CleanSummary,Tokens
9995,Webmaker,[Meta] Solution to user's preference locale se...,meta solution user preference locale setting a...,"[meta, solution, user, preference, locale, set..."
9996,Webmaker,Figure out a way to instantiate variables in a...,figure out way instantiate variables angular i18n,"[figure, out, way, instantiate, variables, ang..."
9997,Webmaker,Change SECRET_SESSION to match other apps,change secret session match other apps,"[change, secret, session, match, other, apps]"
9998,Webmaker,Missing selectize in define details-form,missing selectize define details form,"[missing, selectize, define, details, form]"
9999,Webmaker,Popcorn video from three months ago - Text fie...,popcorn video from three months ago text field...,"[popcorn, video, from, three, months, ago, tex..."


In [5]:
all_words = [word for tokens in data['Tokens'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['Tokens']]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

77422 words total, with a vocabulary size of 10889
Max sentence length is 34


In [6]:
categories = {}
keys = enumerate(data['Product'].unique())
for key, value in keys:
    categories[value] = key + 1
data['ProductIndex'] = data['Product'].map(categories)
print(categories)

# Этот код - пробовал объединять схожие категории, не нужен сейчас
#keys = sorted(list(set(data['Product'])))
#print(keys)
#data['ProductIndex'] = [keys.index(key) for key in data['Product']]

#joinedKeys = {}
#index = 6
#for key in keys:
#    k = key.lower()
#    if "mozilla" in k or "bugzilla" in k or "web" in k:
#        joinedKeys.setdefault(key, 0)
#    elif "firefox" in k:
#        joinedKeys.setdefault(key, 1)
#    elif "penelope" in k or "documentation" in k:
#        joinedKeys.setdefault(key, 2)
#    elif "core" in k or "task" in k or "socorro" in k or "seamonkey" in k or "tool" in k or "develop" in k:
#        joinedKeys.setdefault(key, 3)
#    elif "testing" in k:
#        joinedKeys.setdefault(key, 4)
#    elif "infractructure" in k or "release" in k or "tracking" in k:
#        joinedKeys.setdefault(key, 5)
    # next is for other - too litle data for that
#    elif "community building" in k or "directory" in k or "skywriter" in k or "shield" in k or "nspr" in k \
#            or "nss" in k or "privacy" in k or "snippets" in k or "jss" in k:
#        joinedKeys.setdefault(key, 100)
#    else:
#        joinedKeys.setdefault(key, index)
#        index += 1

#joinedKeysList = list(joinedKeys.keys())
#print(len(joinedKeysList))
#print(len(set(joinedKeys.values())))
#print(joinedKeys)
#data['JoinedProductIndex'] = [joinedKeys[key] for key in data['Product']]
data.to_pickle('dataframe.pkl')
data.tail()

{'Thunderbird': 1, 'Calendar': 2, 'Core': 3, 'Firefox OS': 4, 'www.mozilla.org': 5, 'Toolkit': 6, 'Firefox': 7, 'SeaMonkey': 8, 'Chat Core': 9, 'Firefox for Android Graveyard': 10, 'Mozilla Labs': 11, 'Firefox Build System': 12, 'Tree Management Graveyard': 13, 'Toolkit Graveyard': 14, 'Participation Infrastructure': 15, 'Developer Documentation': 16, 'MailNews Core': 17, 'Mozilla Localizations': 18, 'Webtools': 19, 'Instantbird': 20, 'Cloud Services': 21, 'Firefox for Metro': 22, 'Firefox for Android': 23, 'Firefox Health Report Graveyard': 24, 'Other Applications': 25, 'addons.mozilla.org Graveyard': 26, 'Websites Graveyard': 27, 'support.mozilla.org': 28, 'Firefox Affiliates Graveyard': 29, 'Bugzilla': 30, 'Websites': 31, 'Infrastructure & Operations': 32, 'mozilla.org Graveyard': 33, 'Release Engineering': 34, 'Tracking': 35, 'Firefox for iOS': 36, 'Hello (Loop)': 37, 'Mozilla QA': 38, 'Mozilla QA Graveyard': 39, 'Core Graveyard': 40, 'Tech Evangelism': 41, 'Testing': 42, 'develope

,Product,Summary,CleanSummary,Tokens,ProductIndex
9995,Webmaker,[Meta] Solution to user's preference locale se...,meta solution user preference locale setting a...,"[meta, solution, user, preference, locale, set...",51
9996,Webmaker,Figure out a way to instantiate variables in a...,figure out way instantiate variables angular i18n,"[figure, out, way, instantiate, variables, ang...",51
9997,Webmaker,Change SECRET_SESSION to match other apps,change secret session match other apps,"[change, secret, session, match, other, apps]",51
9998,Webmaker,Missing selectize in define details-form,missing selectize define details form,"[missing, selectize, define, details, form]",51
9999,Webmaker,Popcorn video from three months ago - Text fie...,popcorn video from three months ago text field...,"[popcorn, video, from, three, months, ago, tex...",51


In [7]:
# Код для выкидывания данных по редким категориям - тоже не нужен

#keys = sorted(list(set(data['Product'])))
#print("Number of products with more then 300 records")
#prodCount = 0
#keyCounts = {}
#for key in keys:
#    count = len(data[(data['Product'] == key)])
#    if count > 300:
#        keyCounts[key] = count
#        prodCount +=1
    
#keyCounts = sorted(keyCounts.items(), key=operator.itemgetter(1))
#print(keyCounts)
#print(keyCounts[0][0])
#print(prodCount)

In [8]:
#print(keyCounts.keys())
#for key in keys:
#    if key not in keyCounts.keys():
#        data = data[(data['Product'] != key)]
#        print(key)
        
#data.tail()
        

In [9]:
# Код создания индексов для оставшихся top 10 категорий - эже не нужен, так как ничего не выкидываю

#data = pd.read_pickle('dataframe.pkl')
#topKeys = list(keyCounts.keys())
#print(topKeys)
#data['Top10ProductIndex'] = [topKeys.index(key) for key in data['Product']]
#data.to_pickle('dataframe.pkl')
#data.head()


Создаю сеть

In [2]:
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model

data = pd.read_pickle('dataframe.pkl')
data = data.sample(frac=1).reset_index(drop=True)

all_words = [word for tokens in data['Tokens'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['Tokens']]
max_sentence_length = max(sentence_lengths)
vocabulary = sorted(list(set(all_words)))
words_total=len(all_words)
vocabulary_size=len(vocabulary)
print("%s всего слов, размер словаря %s" % (words_total, vocabulary_size))
print("Макс длина текстов %s" % max_sentence_length)

summaries = data['Tokens'].tolist()
products = data['ProductIndex'].tolist()

products_number=len(list(set(products)))+1 # так как индекса идут с 1, для top10 было с 0 и прибавлять ничего не надо
print("Всего категорий:", products_number)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(summaries)
textSequences = tokenizer.texts_to_sequences(summaries)

textSequences = pad_sequences(textSequences, maxlen=max_sentence_length)

X_train, X_test, y_train, y_test = train_test_split(textSequences, products, random_state=0)
print(len(X_train))
print(len(X_test))
print(X_train[0:5])
print(X_test[0:5])

print('Размерность X_train:', X_train.shape)
print('Размерность X_test:', X_test.shape)

y_train = to_categorical(y_train, products_number)
y_test = to_categorical(y_test, products_number)
print('y_train форма:', y_train.shape)
print('y_test форма:', y_test.shape)

embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 10
batch_size = 100

inputs = Input(shape=(max_sentence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size+1, output_dim=embedding_dim, input_length=max_sentence_length)(inputs)
reshape = Reshape((max_sentence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(max_sentence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(max_sentence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(max_sentence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=products_number, activation='softmax')(dropout)

model = Model(inputs=inputs, outputs=output)

#checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
#                             mode='auto')
#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# А здесь особенность keras - можно использовать и binary  и categorical - но тогда надо правильно задавать metrics
# Вот тут про это подробно
# https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', 'categorical_accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))  # starts training



77422 всего слов, размер словаря 10889
Макс длина текстов 34
Всего категорий: 79


7500
2500
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0  400  238  356    1]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0  969   85  908 2838]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  144  178  316    5  126   17  330    5   40   17  330  404  255
    17   33  330    8  264  171]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    7 2982 5861  487 1275]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0   70
  2110   49   38   28   29  199]]
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     

Train on 7500 samples, validate on 2500 samples
Epoch 1/10


 100/7500 [..............................] - ETA: 2:48 - loss: 0.0677 - binary_accuracy: 0.9873 - categorical_accuracy: 0.0300

 200/7500 [..............................] - ETA: 2:00 - loss: 0.0670 - binary_accuracy: 0.9873 - categorical_accuracy: 0.0450

...

 700/7500 [=>............................] - ETA: 1:40 - loss: 0.0623 - binary_accuracy: 0.9873 - categorical_accuracy: 0.1714

 800/7500 [==>...........................] - ETA: 1:39 - loss: 0.0615 - binary_accuracy: 0.9873 - categorical_accuracy: 0.1788

 900/7500 [==>...........................] - ETA: 1:33 - loss: 0.0606 - binary_accuracy: 0.9873 - categorical_accuracy: 0.1900

1000/7500 [===>..........................] - ETA: 1:32 - loss: 0.0599 - binary_accuracy: 0.9873 - categorical_accuracy: 0.1900

1100/7500 [===>..........................] - ETA: 1:29 - loss: 0.0593 - binary_accuracy: 0.9873 - categorical_accuracy: 0.1900